# Data pre-processing

In [ ]:
import cv2
import os
import numpy as np 
from keras.utils import np_utils
from zipfile import ZipFile


#### The dataset used in this notebook can be downloaded here: 

https://drive.google.com/file/d/1HIo30y-xImvS-OylG8qdZkTQB8-179DT/view?usp=sharing

#### extract images to working directory

In [ ]:
# file_name = "./fmr_dataset2.zip"
# with ZipFile(file_name, 'r') as zipf:
#     zipf.extractall() 


#### parse folder names to categories and map to binary labels

In [ ]:
data_path='./fmr_dataset2'
classes=os.listdir(data_path)
labels=[l for l, c in enumerate(classes)]
classes_labels = dict(zip(classes, labels))
print(classes_labels)

#### load  images to greyscale and resize them

In [ ]:
IM_SIZE = 33
data = []
labels = []

for c in classes:
    class_folder_path=os.path.join(data_path, c)
    im_names=os.listdir(class_folder_path)
        
    for n in im_names:
        im_path = os.path.join(class_folder_path,n)
        im_gray = cv2.imread(im_path, 0)
        im_resized = cv2.resize(im_gray,(IM_SIZE,IM_SIZE))
        data.append(im_resized)
        labels.append(classes_labels[c])

#### normalize the pixel arrays and convert labels to one-hot vectors, save those variables for use with keras

In [ ]:
data = np.array(data)/255.
data = data.reshape((data.shape[0],IM_SIZE,IM_SIZE,1))
labels = np_utils.to_categorical(np.array(labels))

# Training 

In [ ]:
from keras.models import Sequential
from keras.layers import Dense,Activation,Flatten,Dropout, Conv2D,MaxPooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.models import load_model

#### create a cnn 

In [ ]:
model=Sequential()
model.add(Conv2D(200,(3,3),input_shape=data.shape[1:]))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Conv2D(100,(3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Flatten())
model.add(Dropout(0.5))
model.add(Dense(50,activation='relu'))
model.add(Dense(2,activation='softmax'))

In [ ]:
model.summary()

#### compile it

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

#### split the data

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.1)

#### train the cnn

In [ ]:
earlystopper = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='min')
checkpoint = ModelCheckpoint("model.h5", monitor='val_loss', verbose=1, 
                             save_best_only=True, mode='min')
callback_list= [earlystopper, checkpoint]

In [ ]:
training=model.fit(X_train, y_train, epochs=70, callbacks=callback_list, validation_split=0.2)

#### load best model

In [ ]:
model = load_model('model.h5')

# Evaluation

In [ ]:
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

#### plot loss and accuracy on train and dev sets

In [ ]:
plt.plot(training.history['loss'],'r', label='training loss')
plt.plot(training.history['val_loss'], label='validation loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.xticks(rotation='vertical')
plt.title('train and dev loss as function of epoch')
plt.legend()
plt.show()

In [ ]:
plt.plot(training.history['accuracy'],'r',label='training accuracy')
plt.plot(training.history['val_accuracy'],label='validation accuracy')
plt.xlabel('# epochs')
plt.xticks(rotation='vertical')
plt.ylabel('accuracy')
plt.title('train and dev acc as function of epoch')
plt.legend()
plt.show()

#### evaluate the model

In [ ]:
model.evaluate(X_test, y_test)

#### print classification report

In [ ]:
y_pred = model.predict(X_test)
cr = classification_report(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(cr)


#### plot confusion matrix and roc curve

In [ ]:
sns.heatmap(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)), annot=True)
plt.title('confusion matrix heatmap')
plt.show()

In [ ]:
import sklearn.metrics as metrics
preds = y_pred.argmax(axis=1)
fpr, tpr, threshold = metrics.roc_curve(y_test.argmax(axis=1), preds)
roc_auc = metrics.auc(fpr, tpr)

plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


#### launch video capture with real time labeling

please download this xml file: https://drive.google.com/file/d/1x2aaeSL0afHl3GUOlU33CWcY3q2vTh20/view?usp=sharing to your working directory 

In [ ]:
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)
labels_dict={1:'with_mask',0:'without_mask'}
color_dict={0:(0,0,255),1:(0,255,0)}

while True:
    ret, frame = cap.read()
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x, y, w, h) in faces:
        face_img=gray[y:y+w,x:x+w]        
        resized=cv2.resize(face_img,(IM_SIZE,IM_SIZE))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,IM_SIZE,IM_SIZE,1))
        result=model.predict(reshaped)
        label=np.argmax(result,axis=1)[0]
        cv2.rectangle(frame,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(frame,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(frame, labels_dict[label], (x, y-10), cv2.FONT_HERSHEY_SIMPLEX,0.5,(255,255,255),2)
        
    cv2.imshow('LIVE',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
cap.release()
